In [ ]:
import pandas as pd
from pyprojroot import here

import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
df = data.iloc[:,68:73]

In [ ]:
# Hospital level
a = list()

for c in df.columns:
    b = data.groupby(['Country', 'Hospital Name'])[c].value_counts(normalize=True)
    b.name = c
    a.append(b)

d = pd.concat(a, axis=1)

d = d[d.any(axis=1)]

d.index.names = ['Country', 'Hospital Name', 'Categories']

In [ ]:
# Country level
unique_hospitals = d.reset_index().groupby('Country')['Hospital Name'].nunique()
unique_hospitals.name = 'Country'

e = d.groupby(['Country', 'Categories']).sum()
a = list()

for c in e.columns:
    f = e[c].div(unique_hospitals, level='Country')
    f.name = c
    a.append(f)

g = pd.concat(a, axis=1)

In [ ]:
mask = g.index.get_level_values(1)=='Checked'
g = g[mask]
index = g.index.get_level_values(0)
g.index = index

In [ ]:
# EU level
from conversion import population
population = pd.Series(population)
weight = population / population.sum()

m = g.index.get_level_values(0).map(weight)
h = g.multiply(m, axis=0)

In [ ]:
series = ((h.sum() / h.sum().sum()) * 100)

In [ ]:
series.index = series.index.str[55:-1]

In [ ]:
def split_at_middle_word(s):
    words = s.split()
    if len(words) < 2:
        return s  # no split possible
    mid = len(words) // 2
    return " ".join(words[:mid]) + "\n" + " ".join(words[mid:])

series.index = [split_at_middle_word(i) for i in series.index]

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))

series.plot.barh(
    ax=ax, 
    color='lightgrey',
    width=.6)

ax.set_xlim([0,100])

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.savefig(here() / 'output/plots/interventions.png', dpi=300, bbox_inches='tight')